process for 2024 data:
1. first need to combine all images in the same folder, all rgb in one folder, all tiff in one folder and all MAT files from Segmented_by_sam in one folder

# **Mount & Import**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import os
import pandas as pd

# **Tabular Data creation**

get the csv

In [10]:
# Path to the CSV file
csv_path = '/content/drive/Othercomputers/My PC/Thesis/combined_data_2023_updated.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

df.head()

,id,tree,direction,mandarin_number,date,avg_red,avg_green,avg_blue,std_red,std_green,std_blue,height,width,edge_density,tag
0,1_n_1_20230825,1,n,1,20230825,1.173631,3.267353,2.938598,8.405348,22.029888,19.857833,5464,8192,0.137934,0
1,1_n_1_20230929,1,n,1,20230929,23.080680,23.095326,21.652693,48.962308,48.486428,45.493625,5464,8192,0.191935,0
2,1_n_1_20231103,1,n,1,20231103,13.750980,13.507555,13.113424,40.518468,39.679547,38.675582,5464,8192,0.122740,0
3,1_n_1_20231216,1,n,1,20231216,7.626912,12.835019,18.137301,20.289230,29.559554,40.866454,5464,8192,0.232924,0
4,1_n_2_20230825,1,n,2,20230825,2.103231,2.680031,2.149817,15.455984,18.459059,15.112353,5464,8192,0.087966,0


missing values

In [ ]:
# to be written

Categorical

In [15]:
def target_encode_column(df, column_name, target_column):
    """
    Encodes a categorical column based on the mean of the target column.

    Args:
        df (pd.DataFrame): Input DataFrame.
        column_name (str): Name of the categorical column to encode.
        target_column (str): Name of the target column.

    Returns:
        pd.DataFrame: DataFrame with the column encoded.
        dict: Mapping of original categories to their target-based encodings.
    """
    # Compute the mean of the target column for each category
    mapping = df.groupby(column_name)[target_column].mean().to_dict()

    # Replace the column values with their corresponding means
    df[f"{column_name}_encoded"] = df[column_name].map(mapping)

    return df, mapping


In [16]:
def target_encode_categorical_columns(df, columns, target_column):
    """
    Encodes multiple categorical columns based on the mean of the target column.

    Args:
        df (pd.DataFrame): Input DataFrame.
        columns (list): List of categorical columns to encode.
        target_column (str): Name of the target column.

    Returns:
        pd.DataFrame: DataFrame with the encoded columns.
        dict: Dictionary of mappings for each column.
    """
    mappings = {}
    for column in columns:
        df, column_mapping = target_encode_column(df, column, target_column)
        mappings[column] = column_mapping
    return df, mappings


In [19]:
# Function to save target-based mappings as DataFrames in the notebook
def create_target_mappings_dfs(mappings):
    """
    Creates a dictionary of DataFrames for target-based mappings.

    Args:
        mappings (dict): Dictionary of mappings for each column.

    Returns:
        dict: Dictionary of DataFrames with mappings.
    """
    mapping_dfs = {}
    for column, mapping in mappings.items():
        mapping_dfs[column] = pd.DataFrame(list(mapping.items()), columns=[f"{column}_original", f"{column}_mean_tag"])
        print(f"Mapping for '{column}' created as a DataFrame.")
    return mapping_dfs

# Apply target-based encoding
categorical_columns = ['tree', 'direction', 'mandarin_number', 'date']
target_column = 'tag'

# Perform target-based encoding and create mapping DataFrames
df_encoded, target_mappings = target_encode_categorical_columns(df, categorical_columns, target_column)
mapping_dfs = create_target_mappings_dfs(target_mappings)

# Display the mapping DataFrames
for column, mapping_df in mapping_dfs.items():
    print(f"Mapping DataFrame for '{column}':")
    display(mapping_df)


Mapping for 'tree' created as a DataFrame.
Mapping for 'direction' created as a DataFrame.
Mapping for 'mandarin_number' created as a DataFrame.
Mapping for 'date' created as a DataFrame.
Mapping DataFrame for 'tree':


,tree_original,tree_mean_tag
0,1,0.048780
1,2,0.034483
2,3,0.016393
3,4,0.000000
4,5,0.015152
5,6,0.015385
6,7,0.018519
7,8,0.092593
8,9,0.054545
9,10,0.053571


Mapping DataFrame for 'direction':


,direction_original,direction_mean_tag
0,n,0.045077
1,s,0.021472


Mapping DataFrame for 'mandarin_number':


,mandarin_number_original,mandarin_number_mean_tag
0,1,0.051282
1,10,0.000000
2,11,0.000000
3,12,0.000000
4,13,0.333333
5,14,0.000000
6,15,0.000000
7,16,0.000000
8,2,0.049689
9,3,0.031646


Mapping DataFrame for 'date':


,date_original,date_mean_tag
0,20230825,0.000000
1,20230929,0.013333
2,20231103,0.094915
3,20231216,0.048148


In [20]:
# Columns to encode and the target column
categorical_columns = ['tree', 'direction', 'mandarin_number', 'date']
target_column = 'tag'

# Perform target-based encoding
df_encoded, target_mappings = target_encode_categorical_columns(df, categorical_columns, target_column)

# Save the mappings
output_folder = '/content/drive/Othercomputers/My PC/Thesis/target_encoded_mappings'
save_target_mappings(target_mappings, output_folder)

# Display the updated DataFrame
df_encoded.head()


Target-based mapping for 'tree' saved to /content/drive/Othercomputers/My PC/Thesis/target_encoded_mappings/tree_target_mapping.csv
Target-based mapping for 'direction' saved to /content/drive/Othercomputers/My PC/Thesis/target_encoded_mappings/direction_target_mapping.csv
Target-based mapping for 'mandarin_number' saved to /content/drive/Othercomputers/My PC/Thesis/target_encoded_mappings/mandarin_number_target_mapping.csv
Target-based mapping for 'date' saved to /content/drive/Othercomputers/My PC/Thesis/target_encoded_mappings/date_target_mapping.csv


,id,tree,direction,mandarin_number,date,avg_red,avg_green,avg_blue,std_red,std_green,std_blue,height,width,edge_density,tag,tree_encoded,direction_encoded,mandarin_number_encoded,date_encoded
0,1_n_1_20230825,1,n,1,20230825,1.173631,3.267353,2.938598,8.405348,22.029888,19.857833,5464,8192,0.137934,0,0.04878,0.045077,0.051282,0.000000
1,1_n_1_20230929,1,n,1,20230929,23.080680,23.095326,21.652693,48.962308,48.486428,45.493625,5464,8192,0.191935,0,0.04878,0.045077,0.051282,0.013333
2,1_n_1_20231103,1,n,1,20231103,13.750980,13.507555,13.113424,40.518468,39.679547,38.675582,5464,8192,0.122740,0,0.04878,0.045077,0.051282,0.094915
3,1_n_1_20231216,1,n,1,20231216,7.626912,12.835019,18.137301,20.289230,29.559554,40.866454,5464,8192,0.232924,0,0.04878,0.045077,0.051282,0.048148
4,1_n_2_20230825,1,n,2,20230825,2.103231,2.680031,2.149817,15.455984,18.459059,15.112353,5464,8192,0.087966,0,0.04878,0.045077,0.049689,0.000000
